In [1]:
# PYTHON IMPORTS
import os
import copy
from tqdm.notebook import trange, tqdm

# IMAGE IMPORTS 
from PIL import Image
import cv2
import tifffile

# DATA IMPORTS 
import random
import h5py
import netCDF4 as nc
import numpy as np
import glob

# PLOTTING
import matplotlib.pyplot as plt
import matplotlib.cm as cm

# NEURAL NETWORK
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms, datasets
from torchvision.transforms import ToPILImage, GaussianBlur
from torchvision.transforms import Compose, RandomCrop, ToTensor, Normalize
import torch.optim.lr_scheduler as lr_scheduler
import torchvision.models as models

# MY OWN CLASSES
from TileLocator import *

# PREFERENCES
Image.MAX_IMAGE_PIXELS = 933120000

In [2]:
templates_dir = "data/templates/"
tempfiles_dir = "tempfiles/"

boundary_shapefile = f"{templates_dir}HCAD_Harris_County_Boundary.shp"
boundary_points    = f'{tempfiles_dir}boundary_points.shp'
roads_points       = f'{tempfiles_dir}roads_points.shp'

tile_file = f"data/TileIndices/48201CIND0_0992.tif"

roads_fn = f"data/Roads/TexasHighways.shp"
model_checkpoint = "data/TileLocator/checkpoint_071723.pth"

class_names = ["Tiles", "Roads"]

In [13]:
def split_and_run_cnn(image_path, model, tilesize=1024, overhang_size=2):
        
    tensor = transforms.Compose([
        transforms.ToTensor(),
    ])
    
    num_classes = 2
    
    # Load the image
    image = Image.open(image_path)
    
    # Calculate the number of tiles needed
    width, height = image.size
    num_tiles_x = (width + tilesize-1) // tilesize
    num_tiles_y = (height + tilesize-1) // tilesize
    
    # Create an empty list to store the output tiles
    output_tiles = []
    
    output_gen = np.zeros((width, height, num_classes))
    
    # Iterate over each tile
    for tile_x in tqdm(range(num_tiles_x)):
        for tile_y in range(num_tiles_y):
                        
            # Calculate the coordinates for the current tile
            x0 = tile_x * tilesize
            y0 = tile_y * tilesize
            x1 = min(x0 + tilesize, width)
            y1 = min(y0 + tilesize, height)
            
            # Crop the image to the current tile
            tile = image.crop((x0, y0, x1, y1))
            
            # Pad the tile if needed
            pad_width = tilesize - tile.width
            pad_height = tilesize - tile.height
            if pad_width > 0 or pad_height > 0:
                padding = ((0, pad_height), (0, pad_width))
                tile = np.pad(tile, padding, mode='constant')
            
            # Preprocess the tile
            tile = np.array(tile)
            
            #if np.max(tile) == 1:
            #    tile = tile * 255
            
            # tile = np.where(tile > 127, 255, 0).astype(np.uint8)
            
            tile_tensor = tensor(tile).unsqueeze(0).to("cuda")
            
            # Run the CNN on the tile
            output = model(tile_tensor)
            output = output[0, 1:, :, :].cpu().detach().numpy().T
            
            # Store the output tile
            
            x_fin = tilesize - pad_width
            y_fin = tilesize - pad_height
            
            temp = output[0:x_fin, 0:y_fin, :]
            
            temp[:, :overhang_size, :] = 0
            temp[:, overhang_size:, :] = 0
            temp[:, :, overhang_size:] = 0
            temp[:, :, :overhang_size] = 0
            
            output_gen[x0:x1, y0:y1, :] = temp
        torch.cuda.empty_cache()
    return output_gen.T


In [4]:
# Initialize model
# model = torch.load(model_checkpoint)# RectangleClass()
# model = model.to("cuda")
model = RectangleClass(num_classes=3)
checkpoint = torch.load(model_checkpoint)
model.load_state_dict(checkpoint["model_state_dict"])
model = model.to("cuda")

In [6]:
input_folder = f"data/TileIndicesStore/"
output_folder = f"data/TileIndices_Sharpened/"
    
def makeKernel(w):
    kernel = np.ones((w, w)) * 0
    kernel[w // 2, :] = -1
    kernel[:, w // 2] = -1
    kernel[w // 2, w // 2] = kernel.size + np.sum(kernel) + 1
    return kernel
    
if True:
     # Iterate over the files in the input folder
    for filename in tqdm(os.listdir(input_folder)):
        # Check if the file has a supported image extension
        if filename.lower().endswith(('.png', '.jpg', '.jpeg', '.tif', '.tiff')):
            # Open the image file
            image_path = os.path.join(input_folder, filename)
            output_path = os.path.join(output_folder, filename[:-3] + "png")

            image = cv2.imread(image_path, 0)
            
            # Get the original image dimensions
            height, width = image.shape[:2]
            max_size = 14440

            # Determine the scaling factor
            scale = max_size / max(height, width)

            # Calculate the new dimensions
            new_height = int(height * scale)
            new_width = int(width * scale)

            # Resize the image using the calculated dimensions
            image = cv2.resize(image, (new_width, new_height), interpolation=cv2.INTER_CUBIC)
    

            # Create a kernel
            kernel = makeKernel(4)

            # Apply the kernel to the image
            processed_image = cv2.filter2D(image, -1, kernel).astype(np.uint8)
            
            # processed_image = image
            
            # processed_image = cv2.adaptiveThreshold(processed_image, 255,cv2.ADAPTIVE_THRESH_MEAN_C,\
            #                        cv2.THRESH_BINARY,19, 8)

            # Save the processed image to the output folder
            cv2.imwrite(output_path, processed_image.astype(np.uint8))

  0%|          | 0/39 [00:00<?, ?it/s]

In [14]:
input_folder = f"data/TileIndices_Sharpened/"
output_folder = f"data/TileIndicesBoundaries/"
    
 # Iterate over the files in the input folder
for filename in os.listdir(input_folder):
    # Check if the file has a supported image extension
    if filename.lower().endswith(('.png', '.jpg', '.jpeg', '.tif', '.tiff')):
        # Open the image file
        image_path = os.path.join(input_folder, filename)
        output_path = os.path.join(output_folder, filename[:-3] + "tif")
        print(f"Processing {filename} and saved to {output_path}")
        
        # Apply the process function to the image
        processed_image = split_and_run_cnn(image_path, model)
        
        
        processed_image = np.dstack((
            processed_image[0, :, :],
            processed_image[1, :, :],
            np.zeros(processed_image[0, :, :].shape)
        ))
            
        image = processed_image * 255 
        
        cv2.imwrite(output_path, image.astype(np.uint8))

Processing 480233IND0_0382.png and saved to data/TileIndicesBoundaries/480233IND0_0382.tif


  0%|          | 0/15 [00:00<?, ?it/s]

Processing 480287IND0_0281.png and saved to data/TileIndicesBoundaries/480287IND0_0281.tif


  0%|          | 0/10 [00:00<?, ?it/s]

Processing 480287IND0_0288.png and saved to data/TileIndicesBoundaries/480287IND0_0288.tif


...

  0%|          | 0/15 [00:00<?, ?it/s]

Processing 485488IND0_0990.png and saved to data/TileIndicesBoundaries/485488IND0_0990.tif


  0%|          | 0/15 [00:00<?, ?it/s]

Processing 485516IND0_0281.png and saved to data/TileIndicesBoundaries/485516IND0_0281.tif


  0%|          | 0/10 [00:00<?, ?it/s]